<font color="#CA3532"><h1 align="left">Master Data Analytics. EDEM.</h1></font>
<font color="#6E6E6E"><h2 align="left">Herramientas MLOps.</h2></font> 
<font color="#6E6E6E"><h2 align="left">Tarea 1. Pipeline entrenamiento de modelos.</h2></font> 
#### Daniel Ruiz Riquelme
https://docs.metaflow.org/metaflow/basics

##  Install dependencies

In [ ]:
#!pip install metaflow
#!pip install scikit-learn
#!pip install pandas

## Set username

In [13]:
# Set username for workflows
import os
os.environ["USERNAME"] = "adriana"

In [14]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')
    
    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)
        
    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris
    
        # Load the iris dataset
        iris = load_iris()
        
        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member
        
        self.next(self.split_data)

    @step
    def split_data(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=self.random_state)
        self.next(self.train)

    @step
    def train(self):
        # Train the model
        self.model = RandomForestClassifier(max_depth=self.max_depth, n_estimators=self.n_estimators, random_state=self.random_state)
        self.model.fit(self.X_train, self.y_train)    
        self.next(self.show_metrics)


    @step
    def show_metrics(self):
        # Evaluate the model
        y_pred = self.model.predict(self.X_test)
        self.accuracy = accuracy_score(self.y_test, y_pred)
        self.classification_report = classification_report(self.y_test, y_pred)
        
        # Print metrics to logs
        print(f'Accuracy: {self.accuracy}')
        print(f'Classification Report:\n{self.classification_report}')
        
        self.next(self.register_model)
        
    @step
    def register_model(self):
        # Save the model to local storage
        with open('random_forest_model.pkl', 'wb') as f:
            pickle.dump(self.model, f)
        
        self.next(self.end)

    @step
    def end(self):
        pass
    
if __name__ == '__main__':
    TrainingFlow()

Overwriting metaflow_trainingflow.py


In [15]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.12.5 executing TrainingFlow for user:adrianacamposnarvaez
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2024-06-22 12:44:51.037 Workflow starting (run-id 1719053091036109):
2024-06-22 12:44:51.051 [1719053091036109/start/1 (pid 32034)] Task is starting.
2024-06-22 12:44:51.829 [1719053091036109/start/1 (pid 32034)] Task finished successfully.
2024-06-22 12:44:51.839 [1719053091036109/ingest_data/2 (pid 32037)] Task is starting.
2024-06-22 12:44:52.589 [1719053091036109/ingest_data/2 (pid 32037)] Task finished successfully.
2024-06-22 12:44:52.599 [1719053091036109/split_data/3 (pid 32043)] Task is starting.
2024-06-22 12:44:53.362 [1719053091036109/split_data/3 (pid 32043)] Task finished successfully.
2024-06-22 12:44:53.371 [1719053091036109/train/4 (pid 32051)] Task is starting.
2024-06-22 12:44:54.209 [1719053091036109/train/4 (pid 32051)] Task finished successfully.
2024-06-22 12:44:54.219 [1719053